# Benchmark vLLM Server with inference-perf

### Local vLLM Setup using docker

Run vLLM Server as a docker container with the model HuggingFace `HuggingFaceTB/SmolLM2-135M-Instruct`

In [1]:
!export MODEL_NAME="HuggingFaceTB/SmolLM2-135M-Instruct" && \
    docker run --name vllm-server -d --runtime nvidia --gpus all \
    -v ~/.cache/huggingface:/root/.cache/huggingface \
    -p 8000:8000 vllm/vllm-openai:latest \
    --model ${MODEL_NAME}

71c1f998ef3488239cf88c97e0084e6287c87df3f3de3842e47c3751acc43329


**Note**: Configure [vLLM engine arguments](https://docs.vllm.ai/en/latest/serving/engine_args.html#engine-args) like `--max-model-len` and  `--max-num-seqs` according to local compute capacity

### Benchmark with inference_perf

Create a configuration file for the test using `shareGPT` data and run the constant rate test for `30s`. You can also use any of the other data generators like `random`, `shared-prefix` or `synthetic` with their own configuration using the corresponding `config-*.yml` file instead.

In [2]:
!cat config.yml

data:
  type: shareGPT
load:
  type: constant
  rate: 1
  duration: 30
vllm:
  api: chat
  model_name: HuggingFaceTB/SmolLM2-135M-Instruct
  url: http://0.0.0.0:8000

In [3]:
!inference-perf --config_file config.yml

Using configuration from: config.yml
Run started
Run completed


Generating Report ..
total_requests: 38
avg_prompt_tokens: 2.763157894736842
avg_output_tokens: 28.94736842105263
avg_time_per_request: 0.11538009351045873


### Cleanup

Delete vLLM Server docker processes

In [4]:
!docker stop vllm-server && docker rm vllm-server

vllm-server
vllm-server
